In [91]:
#Import dependencies
import pandas as pd


In [92]:
#Read in data
sars_data = pd.read_csv('data/sars_clean.csv')
h1n1_data = pd.read_csv('data/h1n1_clean.csv')
covid_data = pd.read_csv('data/cleancovids.csv')
population_density = pd.read_csv('data/PopulationDensity.csv')

In [93]:
#pull country from each table

sars_country = sars_data[['Country']]
h1n1_country = h1n1_data[['Country']]
covid_country = covid_data[['Country']]

for country in covid_country["Country"]:
    if country == 'Taiwan*':
        country = 'Taiwan'


#append countries into list 
two_of_3 = sars_country.append(h1n1_country)
combined_countries = two_of_3.append(covid_country)


In [94]:
#deduplicate data
countries = []

for row in combined_countries['Country']:
    if row not in countries:
        countries.append(row)

#make list into df
countries_table = pd.DataFrame({"Country" : countries})

In [95]:
#give each country a unique id
countries_table["Country_ID"] = countries_table.reset_index().index
countries_table = countries_table[['Country_ID', 'Country']]

In [96]:
#join tables to give country id

sars_country_code = sars_data.merge(countries_table, how='left', left_on='Country', right_on="Country")
h1n1_country_code = h1n1_data.merge(countries_table, how='left', left_on='Country', right_on="Country")
covid_country_code = covid_data.merge(countries_table, how='left', left_on='Country', right_on="Country")

In [97]:
#drop country from dfs
sars_country_code.drop('Country', axis=1, inplace=True)
h1n1_country_code.drop('Country', axis=1, inplace=True)
covid_country_code.drop('Country', axis=1, inplace=True) 

#add year
sars_country_code["Year"] = '2003'
h1n1_country_code["Year"] = '2009'
covid_country_code["Year"] = '2020'

#add disease id

sars_country_code["Disease_ID"] = '1'
h1n1_country_code["Disease_ID"]= '2'
covid_country_code["Disease_ID"] = '3'


#create disease table

diseases = ["SARS" , "H1N1", "COVID19"]
disease_id = [1,2,3]
disease_table = pd.DataFrame({"Disease_ID" : disease_id, "Disease_NM":  diseases})



In [98]:
##Maniuplate population density for joinability

#Rename column names for ease
population_density.rename(columns={'2003 Density (P/Km²)': '2003', '2009 Density (P/Km²)': '2009', '2020 Density (P/Km²)' : '2020'}, inplace=True)

#create list of years for itteration
years = [2003, 2009, 2020]

#create empty lists to house data
country_data = []
year_data = []
density_data = []

#append each country into list x3 to account for 3 years
for row in population_density['Country']:
    country_data.append(row)
    country_data.append(row)
    country_data.append(row)


#append each year in years list
#length of country list = 480
#430/3 = 160
for row in range(160):
    year_data.append(years[0])
    year_data.append(years[1])
    year_data.append(years[2])

#create dataframe with just year data
population_density_year = population_density[['2003', '2009', '2020']]

#transpose data from columns to rows
population_density_year = population_density_year.transpose()

#itterate through columns to append each countries data into a list in year order
for column in population_density_year:
    pop_density = population_density_year[column]
    
    density_data.append(pop_density[0])
    density_data.append(pop_density[1])
    density_data.append(pop_density[2])

#create dataframe of all data
combined_dataframe = pd.DataFrame({'Country' : country_data, "Year" : year_data, "Density" : density_data})

#join dataframe with countries table to get ids
population_density_table = combined_dataframe.merge(countries_table, how='left', left_on='Country', right_on="Country")

#drop country
population_density_table.drop('Country', axis=1, inplace=True)

#reorganize country table
population_density_table = population_density_table[["Country_ID", "Density", "Year" ]]
population_density_table = population_density_table[population_density_table.Density != '..']

In [99]:
#write each file into csv 
countries_table.to_csv("data/postgres_files/country_table.csv", index= False)
population_density_table.to_csv("data/postgres_files/pop_density_table.csv", index= False)
disease_table.to_csv("data/postgres_files/disease_table.csv", index= False)
sars_country_code.to_csv("data/postgres_files/sars_final.csv", index= False)
h1n1_country_code.to_csv("data/postgres_files/h1n1_final.csv", index= False)
covid_country_code.to_csv("data/postgres_files/covid_final.csv", index= False)

AttributeError: 'DataFrame' object has no attribute 'drop_index'